In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sqlite3 as sql
import folium

%matplotlib inline

In [2]:
hub = pd.read_csv('../Data/hubNashville_2018.csv')

In [3]:
hub.columns.unique

<bound method Index.unique of Index(['Request #', 'Status', 'Request Type', 'Subrequest Type',
       'Additional Subrequest Type', 'Date / Time Opened',
       'Date / Time Closed', 'Request Origin', 'Zipcode', 'Latitude',
       'Longitude'],
      dtype='object')>

In [4]:
hub = hub.rename(columns = {'Request #' : 'request_id', 'Status' : 'status', 'Request Type' : 'request_type', 'Subrequest Type' : 'subrequest_type',
       'Additional Subrequest Type' : 'add_subrequest_type', 'Date / Time Opened' : 'opened',
       'Date / Time Closed' : 'closed', 'Request Origin' : 'origin', 'Zipcode' : 'zipcode', 'Latitude' : 'lat',
       'Longitude' : 'lng'})

In [5]:
focused_hub = hub.loc[~hub['closed'].isna()]

In [6]:
focused_hub['opened'] = pd.to_datetime(focused_hub['opened'])

<ipython-input-6-0af05d51c2fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  focused_hub['opened'] = pd.to_datetime(focused_hub['opened'])


In [7]:
focused_hub['closed'] = pd.to_datetime(focused_hub['closed'])

<ipython-input-7-e1a994f87d3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  focused_hub['closed'] = pd.to_datetime(focused_hub['closed'])


In [8]:
focused_hub['resolution_time'] = (focused_hub.closed-focused_hub.opened)

<ipython-input-8-cc4915dbc075>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  focused_hub['resolution_time'] = (focused_hub.closed-focused_hub.opened)


In [9]:
slow_to_resolve = focused_hub

In [10]:
slow_to_resolve['resolution_time'] = slow_to_resolve['resolution_time'].dt.days

<ipython-input-10-2c61946e436c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slow_to_resolve['resolution_time'] = slow_to_resolve['resolution_time'].dt.days


In [11]:
slow_to_resolve = slow_to_resolve.loc[slow_to_resolve.resolution_time>365]

In [12]:
slow_to_resolve.reset_index()

,index,request_id,status,request_type,subrequest_type,add_subrequest_type,opened,closed,origin,zipcode,lat,lng,resolution_time
0,244,112161,Closed,Property Violations,Residential Complaint,Residential Complaint,2018-10-25 18:28:49,2020-03-18 08:12:19,hubNashville Community,37115.0,36.258170,-86.695750,509
1,250,123468,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,2018-12-14 13:12:04,2020-06-29 14:41:39,Phone,37207.0,36.245086,-86.773350,563
2,427,48380,Closed,"Streets, Roads & Sidewalks",Dip/Bump in Roadway,Dip/Bump in Roadway,2018-02-22 16:38:31,2019-12-30 16:59:17,hubNashville Community,37221.0,36.062642,-86.972435,676
3,522,117153,Closed,Other,Other,Other,2018-11-16 03:45:14,2020-01-17 20:39:58,hubNashville Community,37115.0,36.243802,-86.717873,427
4,577,84291,Closed,Property Violations,Cars Parked on Lawn,Cars Parked on Lawn,2018-07-19 18:50:09,2020-05-08 07:10:49,hubNashville Community,37211.0,36.127623,-86.757789,658
...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,69089,48747,Closed,"Streets, Roads & Sidewalks",Contractor Complaint,Contractor Complaint,2018-02-23 14:08:22,2019-10-31 21:00:15,Phone,37206.0,36.184193,-86.738066,615
283,74106,55691,Closed,Property Violations,Dilapidated Structures,Dilapidated Structures,2018-03-22 21:49:03,2020-06-29 15:46:25,hubNashville Community,37207.0,36.204541,-86.765310,829
284,75385,42789,Closed,"Streets, Roads & Sidewalks",Curbing and Berms,Damaged Curb,2018-01-30 14:50:00,2019-12-30 16:56:00,Phone,37205.0,36.133250,-86.826109,699
285,76156,52022,Closed,"Streets, Roads & Sidewalks",Contractor Complaint,Contractor Complaint,2018-03-07 16:33:27,2019-10-31 21:00:15,hubNashville Community,37206.0,36.176013,-86.751350,603


In [13]:
grouped_slow_requests = slow_to_resolve.groupby(['zipcode', 'request_type'])['request_id'].agg('count').reset_index()

In [14]:
grouped_slow_requests

,zipcode,request_type,request_id
0,37013.0,Other,1
1,37013.0,Planning & Zoning,1
2,37013.0,Property Violations,3
3,37013.0,"Streets, Roads & Sidewalks",1
4,37015.0,Property Violations,2
...,...,...,...
82,37220.0,"Streets, Roads & Sidewalks",1
83,37221.0,Education & Libraries,1
84,37221.0,Planning & Zoning,1
85,37221.0,"Streets, Roads & Sidewalks",3


In [15]:
conn = sql.connect('../Data/metro_survey.db')
cur = conn.cursor()

In [16]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

available_tables = (cur.fetchall())

In [17]:
print(available_tables)

[('codes',), ('equality_and_access',), ('general_services',), ('info',), ('overall',), ('recreation',), ('safety',), ('transportation',)]


In [18]:
safety_df = pd.read_sql("SELECT * FROM safety;", conn)

In [19]:
respondent_info = pd.read_sql("SELECT * FROM Info;", conn)

In [20]:
safety_exp1 = pd.merge(safety_df, respondent_info)

In [21]:
safety_exp1

,Id,Crime Prevention,Police - Overall,Fire and EMS - Overall,Police Visibility,Traffic Enforcement,Police Professionalism,Police Response Time,Fire Services,Fire Response Time,...,ZIP Code,Quality of Life,Overall Feeling of Safety,Neighborhood Safety,Neighborhood Appearance,Nashville as Residence,Nashville as Place to Raise Children,Nashville as Workplace,Nashville as Retirement Place,Gender Identity
0,1,Very Dissatisfied,Very Dissatisfied,Satisfied,Neutral,Neutral,Neutral,Dissatisfied,Neutral,Very Satisfied,...,37208,Neutral,Neutral,Neutral,Satisfied,Neutral,Good,Neutral,Neutral,Female
1,2,Dissatisfied,Satisfied,Very Satisfied,Satisfied,Dissatisfied,Satisfied,Don't Know,Satisfied,Don't Know,...,37211,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Good,Good,Female
2,3,Neutral,Neutral,Satisfied,Neutral,Satisfied,Neutral,Satisfied,Don't Know,Don't Know,...,37208,Very Satisfied,Neutral,Satisfied,Satisfied,Excellent,Excellent,Excellent,Excellent,Male
3,4,Satisfied,Very Satisfied,Don't Know,Satisfied,Satisfied,Very Satisfied,Don't Know,Don't Know,Don't Know,...,37203,Satisfied,Very Satisfied,Satisfied,Satisfied,Excellent,Don't Know,Excellent,Don't Know,Male
4,5,Don't Know,Neutral,Don't Know,Don't Know,Don't Know,Don't Know,Don't Know,Don't Know,Don't Know,...,37138,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Good,Good,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,2167,Neutral,Neutral,Neutral,Neutral,Satisfied,Don't Know,Don't Know,Don't Know,Don't Know,...,37203,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Excellent,Good,Female
2167,2168,Dissatisfied,Satisfied,Satisfied,Neutral,Neutral,Satisfied,Neutral,Satisfied,Satisfied,...,37027,Satisfied,Dissatisfied,Satisfied,Very Satisfied,Good,Good,Good,Good,Female
2168,2169,Neutral,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Neutral,Neutral,Neutral,...,37209,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Good,Neutral,Male
2169,2170,Neutral,Dissatisfied,Satisfied,Dissatisfied,Dissatisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,...,37217,Neutral,Satisfied,Satisfied,Dissatisfied,Good,Good,Good,Good,Male


In [22]:
safety_exp2 = pd.read_sql('SELECT * FROM Safety INNER JOIN Info ON Safety.Id = Info.Id;', conn)

In [23]:
safety_exp2

,Id,Crime Prevention,Police - Overall,Fire and EMS - Overall,Police Visibility,Traffic Enforcement,Police Professionalism,Police Response Time,Fire Services,Fire Response Time,...,ZIP Code,Quality of Life,Overall Feeling of Safety,Neighborhood Safety,Neighborhood Appearance,Nashville as Residence,Nashville as Place to Raise Children,Nashville as Workplace,Nashville as Retirement Place,Gender Identity
0,1,Very Dissatisfied,Very Dissatisfied,Satisfied,Neutral,Neutral,Neutral,Dissatisfied,Neutral,Very Satisfied,...,37208,Neutral,Neutral,Neutral,Satisfied,Neutral,Good,Neutral,Neutral,Female
1,2,Dissatisfied,Satisfied,Very Satisfied,Satisfied,Dissatisfied,Satisfied,Don't Know,Satisfied,Don't Know,...,37211,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Good,Good,Female
2,3,Neutral,Neutral,Satisfied,Neutral,Satisfied,Neutral,Satisfied,Don't Know,Don't Know,...,37208,Very Satisfied,Neutral,Satisfied,Satisfied,Excellent,Excellent,Excellent,Excellent,Male
3,4,Satisfied,Very Satisfied,Don't Know,Satisfied,Satisfied,Very Satisfied,Don't Know,Don't Know,Don't Know,...,37203,Satisfied,Very Satisfied,Satisfied,Satisfied,Excellent,Don't Know,Excellent,Don't Know,Male
4,5,Don't Know,Neutral,Don't Know,Don't Know,Don't Know,Don't Know,Don't Know,Don't Know,Don't Know,...,37138,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Good,Good,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166,2167,Neutral,Neutral,Neutral,Neutral,Satisfied,Don't Know,Don't Know,Don't Know,Don't Know,...,37203,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Excellent,Good,Female
2167,2168,Dissatisfied,Satisfied,Satisfied,Neutral,Neutral,Satisfied,Neutral,Satisfied,Satisfied,...,37027,Satisfied,Dissatisfied,Satisfied,Very Satisfied,Good,Good,Good,Good,Female
2168,2169,Neutral,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,Neutral,Neutral,Neutral,...,37209,Satisfied,Satisfied,Satisfied,Satisfied,Good,Good,Good,Neutral,Male
2169,2170,Neutral,Dissatisfied,Satisfied,Dissatisfied,Dissatisfied,Dissatisfied,Satisfied,Satisfied,Satisfied,...,37217,Neutral,Satisfied,Satisfied,Dissatisfied,Good,Good,Good,Good,Male


In [24]:
list(safety_exp1.columns)

['Id',
 'Crime Prevention',
 'Police - Overall',
 'Fire and EMS - Overall',
 'Police Visibility',
 'Traffic Enforcement',
 'Police Professionalism',
 'Police Response Time',
 'Fire Services',
 'Fire Response Time',
 'Emergency Medical Services (EMS)',
 'EMS Response Time',
 'Fire Professionalism',
 'EMS Professionalism',
 '911 Services',
 'Non-Emergency Number',
 'Period',
 'Date',
 'ZIP Code',
 'Quality of Life',
 'Overall Feeling of Safety',
 'Neighborhood Safety',
 'Neighborhood Appearance',
 'Nashville as Residence',
 'Nashville as Place to Raise Children',
 'Nashville as Workplace',
 'Nashville as Retirement Place',
 'Gender Identity']

In [25]:
list(safety_exp2.columns)

['Id',
 'Crime Prevention',
 'Police - Overall',
 'Fire and EMS - Overall',
 'Police Visibility',
 'Traffic Enforcement',
 'Police Professionalism',
 'Police Response Time',
 'Fire Services',
 'Fire Response Time',
 'Emergency Medical Services (EMS)',
 'EMS Response Time',
 'Fire Professionalism',
 'EMS Professionalism',
 '911 Services',
 'Non-Emergency Number',
 'Id',
 'Period',
 'Date',
 'ZIP Code',
 'Quality of Life',
 'Overall Feeling of Safety',
 'Neighborhood Safety',
 'Neighborhood Appearance',
 'Nashville as Residence',
 'Nashville as Place to Raise Children',
 'Nashville as Workplace',
 'Nashville as Retirement Place',
 'Gender Identity']

In [29]:
safety_exp1 = safety_exp1[['Id', 'Crime Prevention', 'Police - Overall', 'Police Visibility', 'Police Professionalism', 'Police Response Time', 'ZIP Code']]

In [34]:
safety_grouped = safety_exp1.groupby(["ZIP Code", "Police - Overall"])["Id"].count()

In [37]:
safety_grouped.head(25)

ZIP Code  Police - Overall 
37013     Dissatisfied         28
          Don't Know            8
          Neutral              36
          Satisfied            79
          Very Dissatisfied     8
          Very Satisfied       21
37015     Don't Know            1
          Satisfied             1
37023     Neutral               1
37027     Dissatisfied          1
          Don't Know            3
          Neutral               3
          Satisfied            12
          Very Satisfied        3
37072     Don't Know            1
          Neutral               7
          Satisfied            21
          Very Dissatisfied     2
          Very Satisfied        6
37076     Dissatisfied          8
          Don't Know            2
          Neutral              22
          Satisfied            47
          Very Dissatisfied     3
          Very Satisfied       15
Name: Id, dtype: int64

In [47]:
nash_map = folium.Map(location = [36.1612, -86.7775], zoom_start = 11)

In [48]:
nash_map